<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>DBQL Usage</b>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Database Query Logging (DBQL) is used to track query behaviour in Teradata Vantage. Like other system tables, the predefined DBQL logs are created as relational tables in DBC during normal Teradata Database installation. However, while most system tables are populated automatically, you control whether or not to populate the DBQL tables. If you choose not to use the feature, the tables remain empty.
<br>
<p style = 'font-size:16px;font-family:Arial'><b>When to Use DBQL</b>
<p style = 'font-size:16px;font-family:Arial'>Use DBQL to:

<li style = 'font-size:16px;font-family:Arial'>Capture query/statement counts and response times. You can look in the DBQL logs to see if this is the first time this query has been broken and what the historical pattern of its execution has been.</li>
<li style = 'font-size:16px;font-family:Arial'>Validate that nonunique secondary indexes (NUSIs) are actually used. You can free up space and improve table update performance by deleting NUSIs.</li>
<li style = 'font-size:16px;font-family:Arial'>Determine which aggregate join indexes (AJIs) are used the most. This allows you to make a cost-benefit analysis on their AJIs, comparing usage against size and time to build.</li>
<li style = 'font-size:16px;font-family:Arial'>Analyze SQL text and process steps. View the SQL text that inadvertently wiped out the system using DBQL historical data and then attempt to rebuild the lost data.</li>
<li style = 'font-size:16px;font-family:Arial'>Log optimizer queries as XML documents.</li>
<li style = 'font-size:16px;font-family:Arial'>Make further refinements to workload groups and scheduling.</li>
<li style = 'font-size:16px;font-family:Arial'>Discover potential application improvements.</li>
<li style = 'font-size:16px;font-family:Arial'>Understand basic workload trends, such as growth patterns and workload mix, and identify outliers.</li>
<li style = 'font-size:16px;font-family:Arial'>Determine whether the query has redrive protection and if the request was redriven after a database failure.</li>
</p>    
<br>
<p style = 'font-size:16px;font-family:Arial'>DBQL provides a series of predefined tables and views that can store historical records of queries and their duration, performance, and target activity based on rules you specify. DBQL is flexible enough to log information on the variety of SQL requests, from short transactions to longer-running analysis and mining queries that run on Teradata Database. You can log information for an application name, a user or group of users, or a list of accounts.
</p>


<p style = 'font-size:16px;font-family:Arial'><a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Database-Administration/March-2019/Tracking-Query-Behavior-with-Database-Query-Logging-Operational-DBAs'>Link to the DBQL documentation</a></p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Connect to Vantage</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Let's start by connecting to the Vantage system </b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
Set query_band='DEMO=DBQLUsage.ipynb;' update for session;

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>2. Session Details</b></p>
<p style = 'font-size:18px;font-family:Arial'>First let us get the different sessions and Demos executed</p>


In [ ]:
SELECT SessionNO,LogonPENO,LogonTime, l.UserName,queryband,
SUBSTR(queryband,INDEX(queryband,'DEMO=')+5,INDEX(queryband,';')) as DemoName,  PEState, AMPState FROM TABLE ( syslib.monitorsession(-1,'*',0)) AS x 
		inner join dbc.qrylog l ON Sessionid = SessionNO and queryband like '%DEMO=%' order by AMPState;

<p style = 'font-size:18px;font-family:Arial'>From the above output please select a session for which we want to go ahead with the further analysis.</p>
<br>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. DBQL Log Table</b></p>
<p style = 'font-size:16px;font-family:Arial'>Replace the <b>SessionIdParam</b> in the below query with the session we want to analyze further. </p>

In [ ]:
select sessionid, queryband,starttime, querytext, NumSteps,AmpCPUTime, TotalIOCount,
cast((firstresptime - starttime second(4)) as decimal(8,2)) as RunSecs from dbc.qrylog 
		where sessionid=2501 ---SessionIdParam 
    order by RunSecs DESC;

<p style = 'font-size:18px;font-family:Arial'>The above output shows all queries executed in the Session you have selected. The first query is the query taking highest execution time and the last query is the query taking least time to execute.</p>
<br>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. NOS fields in DBQL log table</b></p>
<p style = 'font-size:16px;font-family:Arial'>The dbc.qrylog view has many other columns which can be used for other Analysis. Like the NOS columns as shown in the query below.</p>
<p style = 'font-size:16px;font-family:Arial'>Replace the <b>SessionIdParam</b> in the below query with the same sessionId we used in the above query. </p>

In [ ]:
Select SessionId,starttime,NosRecordsSkipped,NosMaxIOWaitTime,NosRecordsReturnedKB,NosPhysReadIO,NosFilesSkipped,
NosTables,NosTotalIOWaitTime,NosCPUTime,NosRecordsReturned,NosPhysReadIOKB,NosFiles from dbc.dbqlogtbl 
where sessionid=2501 ---SessionIdParam
order by 2,1;

<p style = 'font-size:18px;font-family:Arial'>The above output shows all queries executed in the Session using the NOS functionality. </p>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. DBQL log table Details</b></p>
<p style = 'font-size:16px;font-family:Arial'>The dbc.qrylog view is a view on the dbqlogtbl which has many different columns. Default rows are stored in the DBC.DBQLogTbl table, the foundation of the DBQL feature. The default is one default row per query.</p>

<p style = 'font-size:16px;font-family:Arial'><b>The Default Row</b></p>
<p style = 'font-size:16px;font-family:Arial'>The fields of the default row provide general query information that is usually adequate for investigating a query that is interfering with performance. When no options are specified, a default row includes:</p>

<li style = 'font-size:16px;font-family:Arial'>User ID and user name under which the session being logged was initiated</li>
<li style = 'font-size:16px;font-family:Arial'>Unique ID for the process, session, and host (client) connection</li>
<li style = 'font-size:16px;font-family:Arial'>Account string, expanded as appropriate, that was current when the query completed</li>
<li style = 'font-size:16px;font-family:Arial'>First 200 characters of the query SQL statement</li>
<li style = 'font-size:16px;font-family:Arial'>CPU and I/O statistics</li>
<li style = 'font-size:16px;font-family:Arial'>Default database name that was current when the query completed</li>
</p>
<p style = 'font-size:16px;font-family:Arial'>Please find the details of the columns in the DBQL table <a href = 'https://docs.teradata.com/r/Teradata-Database-Administration/April-2018/Tracking-Query-Behavior-with-Database-Query-Logging-Operational-DBAs/Main-Log-Table-DBQLogTbl'>here</a>. Execute the below query to check the column details from the DBQLogTbl</a></p>

In [ ]:
select ColumnName, ColumnFormat, ColumnType, ColumnLength from dbc.ColumnsV where tablename = 'DBQLogtbl' order by ColumnName

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Other DBQL tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Apart from the DBQLogTbl there are other DBQL tables which can be used for complete analysis of the system. The DBQL tables are created in database DBC by the DIP utility during installation. For details, see “Database Initialization Program (DIP)” in Teradata Vantage™ - Database Utilities. The tables are empty until you run a BEGIN QUERY LOGGING or REPLACE QUERY LOGGING request with the required option, as shown in the following table.</p>

<table style ='font-size:14px;font-family:Arial'> <tr><th>Dictionary Table</th>
    <th>Purpose</th>
    <th>Method Used To Populate</th>
  </tr>
  <tr>
    <td>DBQLExplainTbl</td>
    <td>Contains the explain information in an unformatted string without line breaks.</td>  
    <td>The EXPLAIN option.</td>
  </tr>
  <tr>
    <td>DBQLogTbl</td>
    <td>Is the main table containing information about queries being logged.</td>  
    <td>A BEGIN QUERY LOGGING or REPLACE QUERY LOGGING request. No option is required.</td>
  </tr>
  <tr>
    <td>DBQLObjTbl</td>
    <td>Stores information on the target objects of the query being logged. One row is logged for each object referenced by the query.</td>  
    <td>The OBJECT option.</td>
  </tr>
  <tr>
    <td>DBQLParamTbl</td>
    <td>Logs the parameter variable name, type, position, and value for each parameter in a parameterized request. This table contains confidential user information and should only be accessed by personnel trusted with the DBC password.</td>  
    <td>The PARAMINFO option.</td>
  </tr>
  <tr>
    <td>DBQLRuleCountTbl</td>
    <td>Stores the cardinality of DBQLRuleTbl (for internal use only).</td>  
    <td>A BEGIN QUERY LOGGING request.</td>
  </tr>  
  <tr>
    <td>DBQLRuleTbl</td>
    <td>Stores the rules resulting from each BEGIN QUERY LOGGING or REPLACE QUERY statement.
One row exists for each set of username, account string, and application name specified in the BEGIN QUERY LOGGING and REPLACE QUERY LOGGING statements.

The END QUERY LOGGING statement removes rows from the rule table.

This table is for internal use only.</td>  
    <td>A BEGIN QUERY LOGGING or REPLACE QUERY LOGGING request.</td>
  </tr>
  <tr>
    <td>DBQLSQLTbl</td>
    <td>Stores the full SQL text of the query. One query string may require more than one row.</td>
    <td>The WITH SQL option.</td>
  </tr>
  <tr>
    <td>DBQLStepTbl</td>
    <td>Stores information about each processing step used to satisfy the query. One row is logged for each step, including parallel steps.</td>  
    <td>The WITH STEPINFO option.</td>
  </tr> 
  <tr>
    <td>DBQLSummaryTbl</td>
    <td>Stores information about queries that meet the criteria for a rule specifying the LIMIT SUMMARY or LIMIT THRESHOLD option.</td>
    <td>The LIMIT SUMMARY or LIMIT THRESHOLD option.</td>
  </tr>
  <tr>
    <td>DBQLUtilityTbl</td>
    <td>Each row stores information about one completed load/export or Data Stream Architecture job.</td>  
    <td>The WITH UTILITYINFO option.</td>
  </tr>
  <tr>
    <td>DBQLXMLLockTbl</td>
    <td>Logs lock delays from the Lock Manager in the AMPs in XML format.</td>
    <td>The WITH LOCK option.</td>
  </tr>
  <tr>
    <td>DBQLXMLTbl</td>
    <td>Stores the query plan for all DML statements and statistics usage information in an XML document.</td>  
    <td>The WITH XMLPLAN option or the WITH [DETAILED] STATSUSAGE option. You can also specify these options together.</td>
  </tr> 
</table>

<p style = 'font-size:16px;font-family:Arial'>Please find the details of the DBQL tables <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Database-Administration/March-2019/Tracking-Query-Behavior-with-Database-Query-Logging-Operational-DBAs/DBQL-Tables'>here.</a></p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>6. Check Query performance using DBQL log table</b></p>
<p style = 'font-size:16px;font-family:Arial'>The DBQL table is used to check the performance of the queries. Let's check the queries from the session we are analyzing above.</p>

In [ ]:
select sessionid, queryid, queryband,starttime, querytext, NumSteps,AmpCPUTime, TotalIOCount,
cast((firstresptime - starttime second(4)) as decimal(8,2)) as RunSecs from dbc.qrylog 
where sessionid=2501 ---SessionIdParam 
order by Runsecs desc; 

<p style = 'font-size:18px;font-family:Arial'>The above output shows all queries executed in the Session you have selected. The first query is the query taking highest execution time and the last query is the query taking least time to execute.</p>
<br>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>7. DBQL Step table usage</b></p>
<p style = 'font-size:16px;font-family:Arial'>We will further continue the analysis by checking the step details of the longest running query by replacing the parameter <b>SampleQueryId</b> with the actual QueryId from the output above.</p>

In [ ]:
select
ql.queryid, qls.StepStartTime,
CAST( EXTRACT (SECOND FROM FirstRespTime) + (EXTRACT (MINUTE FROM FirstRespTime) * 60 ) + (EXTRACT (HOUR FROM FirstRespTime) *60*60 ) + (86400 * (CAST ( FirstRespTime AS DATE) - CAST ( starttime AS DATE) ) )
- (EXTRACT (SECOND FROM starttime) + (EXTRACT (MINUTE FROM starttime) * 60 ) + (EXTRACT (HOUR FROM starttime) *60*60 ) ) AS INTEGER) QryRunSecs , 
CAST(ql.AmpCpuTime AS DECIMAL(18,3)) QryCpu , 
ql.totaliocount  QryIO,
ql.spoolusage Qryspool,
qls.StepLev1Num + .1 * qls.StepLev2Num (Decimal(4,1)) as sNum,
qls.StepName as Stp,


/* run time metrics */
qls.EstProcTime,
CAST( EXTRACT (SECOND FROM StepStopTime) + (EXTRACT (MINUTE FROM StepStopTime) * 60 ) + (EXTRACT (HOUR FROM StepStopTime) *60*60 ) + (86400 * (CAST ( StepStopTime AS DATE) - CAST ( StepStartTime AS DATE) ) )
- (EXTRACT (SECOND FROM StepStartTime) + (EXTRACT (MINUTE FROM StepStartTime) * 60 ) + (EXTRACT (HOUR FROM StepStartTime) *60*60 ) ) AS INTEGER)  StepSecs,

/* cpu metrics */
 ZEROIFNULL(CAST(qls.EstCPUCost AS DECIMAL(38,1)) ) EstCpu , 
 CAST(qls.CPUTIME AS DECIMAL(38,1)) ActCpu,
CAST( (qls.maxampcputime * qls.numofactiveamps) AS DECIMAL(38,1))  StepImpCpu , 

/* IO metrics */
ZEROIFNULL(qls.EstIOCost) EstIO,
qls.IOcount
, qls.MaxAmpIO* qls.NumOfActiveAmps StepImpIO ,  

/* PJI metrics */
ZEROIFNULL(CAST ( qls.CPUTIME * 1000.0 / NULLIFZERO(qls.IOCOUNT ) AS DECIMAL(38,1) ) )StepPJI,

/* Spool metrics */
qls.SpoolUsage/1000000 StepSpool_M , 
qls.MaxAmpSpool * qls.NumOfActiveAmps / 1000000 StepImpSpoolMB , 

/* PE  metrics */
ZEROIFNULL(ActCpu/NULLIFZERO(StepImpCpu)) StepPE,

/* row estimates  */  
qls.EstRowCount,
qls.RowCount,
qls.RowCount/nullifzero(qls.EstRowCount) as RowVsEst,
qls.RowsWComprColumns/nullifzero(qls.RowCount) as CompPct,

/* Skew metrics */
(1- qls.CpuTime/ nullifzero(qls.NumOfActiveAmps*qls.maxampcputime)) * 100  as CpuSkewPct, 
(1- qls.IOCount/ nullifzero(qls.NumOfActiveAmps*qls.maxampio)) * 100  as IOSkewPct, 
(1-qls.spoolusage/nullifzero(qls.NumOfActiveAmps*qls.MaxAmpSpool)) * 100 as SpoolSkewPct,
qls.numcombinedpartitions

FROM dbc.dbqlogtbl ql
INNER JOIN dbc.dbqlsteptbl qls
ON  ql.queryid = qls.queryid
AND ql.procid = qls.procid
WHERE ql.queryid = 307191304473102746 ---SampleQueryId
order by qls.queryid,sNum                ;

<p style = 'font-size:18px;font-family:Arial'>The above output shows all the steps for the query we are analyzing. Further to check the details of each step we can check the explain plan of the query using the DBQLExplainTbl as shown in the query below .</p>
<br>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>8. DBQL Explain table usage</b></p>
<p style = 'font-size:16px;font-family:Arial'>Run the below query by replacing the parameter <b>SampleQueryId</b> with the actual QueryId from the output above.</p>

In [ ]:
Select QueryId, ExplainText from dbc.dbqlexplaintbl where queryid = 307191304473102746 ---SampleQueryId
;

<p style = 'font-size:18px;font-family:Arial'>Double Click on the cell showing the Explain Text to check the complete Explain Plan of the Query.</p>
<br>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>9. Information on other DBQL Tables and other Documentation Links</b></p>
<p style = 'font-size:16px;font-family:Arial'>Apart from these main tables like the DBQL log, Step and Explain tables, other DBQL tables can also be used similarly to check the overall performance of the system. Click <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Database-Administration/March-2019/Tracking-Query-Behavior-with-Database-Query-Logging-Operational-DBAs/DBQL-Tables'>here</a> for additional Information on different DBQL Tables,</p>
    
<p style = 'font-size:16px;font-family:Arial'>Below are some links for the documents</p>

<table style ='font-size:14px;font-family:Arial'> 
  <tr>
    <th>Link</th>
    <th>Description</th>
    
  </tr>
  <tr>
    <td><a href = 'https://docs.teradata.com/'>TD Docs (HTML)</a></td>
    <td>Teradata documentation (HTML).</td>  
    
  </tr>
  <tr>
    <td><a href = 'http://www.info.teradata.com'>TD Docs (PDF)</a></td>
    <td>Teradata documentation (HTML).</td>  
    
  </tr>
  <tr>
    <td><a href = 'https://access.teradata.com'>TD Docs One Stop Source</a></td>
    <td>One stop source for Teradata community, product information, and software downloads.
Log in for customer access to:
        Support
        Software updates
        Knowledge articles
        Orange Books</td>  
    
  </tr>
  <tr>
    <td><a href = 'http://www.teradata.com/products-and-services/TEN'>TEN</a></td>
    <td>Teradata education network.</td>  
    
  </tr>
  <tr>
    <td><a href = 'https://community.teradata.com'>Teradata Community</a></td>
    <td>Link to Teradata community (also available from the customer portal).</td>  
    
  </tr>  
</table>


<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>